In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)      

df = pd.read_excel("/kaggle/input/market-basket-analysis/Assignment-1_Data.xlsx")
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit
def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit
def retail_data_prep(dataframe):
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe

df = retail_data_prep(df)

df.describe().T


,count,mean,min,25%,50%,75%,max,std
Quantity,519551.0,9.39742,1.0,1.0,3.0,10.0,248.5,21.281261
Date,519551,2011-07-04 16:03:31.051080704,2010-12-01 08:26:00,2011-03-28 10:52:00,2011-07-20 11:55:00,2011-10-19 15:08:00,2011-12-09 12:50:00,NaN
Price,519551.0,3.32647,0.001,1.25,2.08,4.13,41.94,3.87738
CustomerID,387985.0,15317.042994,12346.0,13950.0,15265.0,16837.0,18287.0,1721.813298


In [2]:
df_fr = df[df['Country'] == "France"]

df_fr.groupby(['BillNo', 'Itemname']).agg({"Quantity": "sum"}).unstack().fillna(0).iloc[0:5, 0:5]

Quantity                                                                                                                   
Itemname 10 COLOUR SPACEBOY PEN 12 COLOURED PARTY BALLOONS 12 EGG HOUSE PAINTED WOOD 12 MESSAGE CARDS WITH ENVELOPES 12 PENCIL SMALL TUBE WOODLAND
BillNo                                                                                                                                            
536370                      0.0                        0.0                       0.0                             0.0                           0.0
536852                      0.0                        0.0                       0.0                             0.0                           0.0
536974                      0.0                        0.0                       0.0                             0.0                           0.0
537065                      0.0                        0.0                       0.0                             0.0                           0.0
537463                      0.0                        0.0                       0.0                             0.0                           0.0

In [3]:
fr_inv_pro_df=df_fr.groupby(['BillNo', 'Itemname']). \
                agg({"Quantity": "sum"}). \
                unstack(). \
                fillna(0). \
                applymap(lambda x: 1 if x > 0 else 0)

frequent_itemsets = apriori(fr_inv_pro_df.astype("bool"),   
                            min_support=0.01,
                            use_colnames=True)


frequent_itemsets.sort_values("support", ascending=False).head()

,support,itemsets
330,0.765306,"((Quantity, POSTAGE))"
332,0.188776,"((Quantity, RABBIT NIGHT LIGHT))"
371,0.181122,"((Quantity, RED TOADSTOOL LED NIGHT LIGHT))"
320,0.170918,"((Quantity, PLASTERS IN TIN WOODLAND ANIMALS))"
315,0.168367,"((Quantity, PLASTERS IN TIN CIRCUS PARADE))"
